# Colab librerias

In [1]:
# --- Detectar si estamos en Google Colab ---
def running_in_colab() -> bool:
    try:
        import google.colab  # solo existe en Colab
        return True
    except ImportError:
        return False

# Instalar dependencias si estamos en Colab
if running_in_colab():
    # pyvista[jupyter] es suficiente; backend html funciona estable
    %pip install -q "pyvista[jupyter]"
    pv_backend = "html"
else:
    pv_backend = "trame"

print("Usando backend:", pv_backend)

Usando backend: trame


# Importacion de librerias y creacion de variables

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pyvista as pv

# Use the Trame backend for interactive inline plots
pv.set_jupyter_backend(pv_backend)

n = 1 # Number of stacks of Miller indexes in each direction
c = 1.0 # Height of the unit cell "z" direction
a = 1.0 # Lattice constant

rhombohedral_angle = np.pi / 3 # radians

lattice_vectors = np.array([
    [a, 0, 0],
    [a * np.cos(rhombohedral_angle), a * np.sin(rhombohedral_angle), 0],
    [0, 0, c]
], dtype=float) # Rhombic cell

base_fractional = np.array([
    [0,   0,   0],
    [1/2, 1/2, 1],  # Este realmente no hace parte de la base pero lo pongo para que se vea el átomo de arriba, los de abajo son para brillouin
    [1/2, 1/2, 0], # Este si hace parte de la base
    [-1/2, 1/2, 0],
    [-1/2, -1/2, 0],
    [1/2, -1/2, 0]
], dtype=float) # Basis atoms positions

base = base_fractional[:3] @ lattice_vectors  # Coordenadas de los átomos en la celda unitaria

# Define Miller index to work with
Miller_indexes = np.stack(np.mgrid[0:n+1, 0:n+1, 0:n+1], axis=-1).astype(float)  # (n+1, n+1, n+1, 3) Miller indices grid
Miller_indexes = Miller_indexes.reshape(-1,3)[1:]  # Convertir a un array n^3 x 3

# Construcción de malla
I, J, K = np.mgrid[0:n+1, 0:n+1, 0:n+1]

# Combinación lineal increible
lattice = I[..., None] * lattice_vectors[0] + J[..., None] * lattice_vectors[1] + K[..., None] * lattice_vectors[2]
lattice = lattice.reshape(-1,3)[1:] # Al poner -1, numpy infiere la cantidad de puntos (2n+1)^3

print(np.shape(base))

(3, 3)


# Grafica de la celda

In [3]:
def puntos_grafica(base, color, label, lattice_vectors):
    lengths = np.linalg.norm(lattice_vectors, axis=1)
    r = np.min(lengths) / 10  # Radio de las esferas (átomos)
    for i in range(np.size(base, 0)):  # Iterar sobre cada átomo en la base
        # Crear una esfera pequeña para representar un punto
        punto = pv.Sphere(radius=r, center=base[i])  # Centro en la posición del átomo
        # Añadir la esfera con un color específico y un nombre para la leyenda
        plotter.add_mesh(
            punto,
            color=color,
            label=label
        )

# Crear el plotter
plotter = pv.Plotter(notebook=True)

puntos_grafica(base, "green", "Nan", lattice_vectors)
puntos_grafica(lattice, "green", "Nan", lattice_vectors)

# Añadir la leyenda
plotter.add_legend()

# Mostrar
plotter.show(jupyter_backend=pv_backend)

Widget(value='<iframe src="http://localhost:3538/index.html?ui=P_0x1ce4d3012b0_0&reconnect=auto" class="pyvist…

# Principales Simetrías

Este sistema en 2D unicamente deberia tratarse como una red triangular con base en (0, 0), (1/2, 0) y (0, 1/2). Esto debido a que no existe una red de brave romboedrica, por lo tanto se debe tratar como una rectangular.
El sistema 2D como se plantea en el punto es $C_{2V}$, esto debido a que tiene dos planos de espejo $XZ$ y $YZ$, ademas el vector de rotacion $\hat{z}$ es coplanar. Estos planos de espejo se encuentran justamente en las rectas que pasan por la red en $x = 0$ y $y = 0$.

# Grafica celda reciproca

In [4]:
# Calculate reciprocal lattice vectors
reciprocal_vectors = np.zeros((3, 3), dtype=float) # To store reciprocal lattice vectors
for i in range(3):
    reciprocal_vectors[i] = 2 * np.pi * np.cross(lattice_vectors[np.mod(i+1, 3)], lattice_vectors[np.mod(i+2, 3)])  / np.linalg.det(lattice_vectors)

lattice_reciprocal = I[..., None] * reciprocal_vectors[0] + J[..., None] * reciprocal_vectors[1] + K[..., None] * reciprocal_vectors[2]
lattice_reciprocal = lattice_reciprocal.reshape(-1,3)[1:] # Al poner -1, numpy infiere la cantidad de puntos (2n+1)^3

base_reciprocal = base_fractional @ reciprocal_vectors  # Coordenadas de los átomos en la celda unitaria

# Crear el plotter
plotter = pv.Plotter(notebook=True)

puntos_grafica(base_reciprocal[:3], "blue", "Nan", lattice_reciprocal)
puntos_grafica(lattice_reciprocal, "blue", "Nan", lattice_reciprocal)

# Añadir la leyenda
plotter.add_legend()

# Mostrar
plotter.show()

Widget(value='<iframe src="http://localhost:3538/index.html?ui=P_0x1ce4dc03b10_1&reconnect=auto" class="pyvist…

# Brillouin

In [5]:
I, J, K = np.mgrid[-n:n+1, -n:n+1, -n:n+1]



# Combinación lineal increible
pts = I[..., None] * reciprocal_vectors[0] + J[..., None] * reciprocal_vectors[1] + K[..., None] * reciprocal_vectors[2]
pts = pts.reshape(-1,3) # Al poner -1, numpy infiere la cantidad de puntos (2n+1)^3

pts = np.concatenate((pts, base_reciprocal[1:]), axis=0)

# Ordenamiento y eliminacion de puntos innecesarios
mag = np.linalg.norm(pts, axis=1)
order = np.argsort(mag)

k = 6 # Numero de planos a graficar (excluyendo el origen) probar con 6, 8, 10, 12 y 14, entonces toca cambiarlo a mano segun el angulo
pts_ord = pts[order][1:k + 1] # Tomar los primeros puntos (excluyendo el origen) 

rhombohedral_angle_brillouin = np.arccos(np.cos(rhombohedral_angle) / ( 1 + np.cos(rhombohedral_angle) ))
print("Angulo entre los vectores de la base en el espacio recíproco teorico (coseno): ", np.degrees(rhombohedral_angle_brillouin))

print("Puntos ordenados:")
print(pts_ord)

# Crear plotter
plotter = pv.Plotter()

# Crear los planos
#Trucazo increible usando el parametro c_res
for i in range(len(pts_ord)): # Solo los planos hexagonales

    # Rotar los planos para que queden bien orientados los cortes
    if np.isclose(pts_ord[i,2], 0.0):
        ang = 45
    else:
        ang = np.degrees(rhombohedral_angle_brillouin)

    plane = pv.Disc(center=pts_ord[i,:]/2,      # punto central del plano
                     normal=pts_ord[i,:],   # vector normal al plano
                     inner=0.0, outer=5,   # tamaño del plano en direcciones radiales
                     r_res=1, c_res=4)  # densidad de subdivisión
    # Lo intente rotar con esto y nada, no se como rotar los planos para que queden bien orientados
    plane.rotate_vector(vector=pts_ord[i,:], angle=ang, point=pts_ord[i,:]/2)

    plotter.add_mesh(plane, color="aquamarine", opacity=0.5, show_edges=True)

# Mostrar
plotter.show()

Angulo entre los vectores de la base en el espacio recíproco teorico (coseno):  70.52877936550931
Puntos ordenados:
[[ 3.14159265  1.81379936  0.        ]
 [-3.14159265 -1.81379936  0.        ]
 [ 0.          0.         -6.28318531]
 [ 0.          0.          6.28318531]
 [-3.14159265  5.44139809  0.        ]
 [ 3.14159265 -5.44139809  0.        ]]


Widget(value='<iframe src="http://localhost:3538/index.html?ui=P_0x1ce4de0da90_2&reconnect=auto" class="pyvist…